In [62]:
from datetime import datetime
from itertools import combinations, product
from ipywidgets import widgets, interact, Layout
from IPython.core.display import display, HTML

In [83]:
def convert_time(string):
    time = datetime.strptime(string, '%H:%M').time()
    return time

In [84]:
#defines a course and sets variables based on inputs
class Course:
    def __init__(self, name, professor, times, lab):
        self.subject = name[:3]
        self.number = name[-3:]
        self.professor = professor
        self.days = [day for day in times]
        self.times = times
        self.lab = lab
        for x in self.times:
            for b in range(len(self.times[x])):
                self.times[x][b] = convert_time(self.times[x][b])

In [85]:
class Lab:
    def __init__(self, name, professor, times):
        self.subject = name[:3]
        self.number = name[-3:]
        self.professor = professor
        self.days = [day for day in times]
        self.times = times
        for x in self.times:
            for b in range(len(self.times[x])):
                self.times[x][b] = convert_time(self.times[x][b])

In [86]:
#checks if two courses' days and times overlap
def check_overlap(a, b):
    for x,y in product(a.days, b.days):
        if x == y:
            if b.times[x][0] < a.times[x][0]:
                a, b = b, a
            if a.times[x][1] > b.times[x][0]:
                return True
    else:
        return False

In [87]:
#checks if course is already in schedule
def check_repeat(a, b):
    return a.subject + a.number == b.subject + b.number

In [88]:
#checks if course is before user's earliest time
def check_early(a, early_time):
    for day in a.times:
        if a.times[day][0] < convert_time(early_time):
            return True

In [89]:
#checks if course if after user's lastest time
def check_late(a, late_time):
    for day in a.times:
        if a.times[day][0]  > convert_time(late_time):
            return True

In [90]:
#if the user does not want courses on friday, checks if course is on friday
def check_friday(a, friday_option):
    if friday_option:
        return "F" in a.days

In [91]:
#checks if the course's duration in minutes is longer than user's max class duration time
def check_duration(a, max_duration):
    if max_duration != None:
        for day in a.times:
            duration = (a.times[day][1].hour - a.times[day][0].hour)*60 + a.times[day][1].minute - a.times[day][0].minute + (a.times[day][1].second - a.times[day][0].second)/60.0
            if duration > max_duration:
                return True

In [93]:
def check_lab(option):
    labs_needed = []
    for a in option:
        if a.lab == True:
            for b in labs:
                for c in option:
                    if check_overlap(b, c):
                        return False, option
            else:
                labs_needed.append(b)
    else:
        if len(labs_needed) > 0:
            new_options = labs_needed
            for x in option:
                new_options.append(x)
            option = tuple(new_options)
        return True, option

In [94]:
def run_checks(a, b, early_time, late_time, friday_option, max_duration):
    checks = [
        check_overlap(a,b),
        check_repeat(a, b),
        check_early(a, early_time),
        check_late(a, late_time),
        check_friday(a, friday_option),
        check_duration(a, max_duration)
    ]
    if True in checks:
        return True

In [95]:
#prints schedule in txt format
def print_schedule(option):
    for x in range(2):
        print("="*24)
    for week_day in ["M", "T", "W", "R", "F"]:
        schedule = []
        for course in option:
            for day in course.days:
                if day == week_day:
                    schedule.append([str(course.subject + course.number + " " + str(course.times[day][0]) + "-" + str(course.times[day][1])), course.times[day][0]])
        schedule.sort(key=lambda x: x[1])
        if len(schedule) > 0:
            print("___________"+week_day+"____________")
        for course in schedule:
            print(course[0])

In [134]:
style = {'description_width': 'initial'}
layout = Layout(width='40%')

display(HTML('<h1>College Schedule Builder:</h1>'))

inputText = widgets.Text(layout = layout)
print("Courses you can take:")
display(inputText)

no_widget = widgets.IntSlider(description="# of Courses:", min=2, max = 8, step = 1, layout = layout, style = style)
display(no_widget)

start_widget = widgets.IntSlider(description="Earliest Start: (am)", min=6, max = 15, step = 1, layout = layout, style = style)
display(start_widget)

end_widget = widgets.IntSlider(description="Latest End: (pm)", min=14, max = 22, step = 1, layout = layout, style = style)
display(end_widget)

duration_widget = widgets.IntSlider(description="Max Duration: (min)", min=45, max = 360, step = 1, layout = layout, style = style)
display(duration_widget)

friday_widget = widgets.Dropdown(description="Fridays?", options = [True, False], style = style, layout = layout)
display(friday_widget)


Courses you can take:


Text(value='', layout=Layout(width='40%'))

IntSlider(value=2, description='# of Courses:', layout=Layout(width='40%'), max=8, min=2, style=SliderStyle(de…

IntSlider(value=6, description='Earliest Start: (am)', layout=Layout(width='40%'), max=15, min=6, style=Slider…

IntSlider(value=14, description='Latest End: (pm)', layout=Layout(width='40%'), max=22, min=14, style=SliderSt…

IntSlider(value=45, description='Max Duration: (min)', layout=Layout(width='40%'), max=360, min=45, style=Slid…

Dropdown(description='Fridays?', layout=Layout(width='40%'), options=(True, False), style=DescriptionStyle(des…

In [143]:
classes = [
    Course("MAR357", "", {"T": ["17:00", "18:20"], "R": ["17:00", "18:20"]}, False),
    Course("MAR444", "", {"T": ["12:30", "13:50"], "R": ["12:30", "13:50"]}, False),
    Course("MAR444", "", {"T": ["15:30", "16:50"], "R": ["15:30", "16:50"]}, False),
    Course("ACC252", "", {"T": ["11:00", "12:20"], "R": ["11:00", "12:20"]}, False),
    Course("ACC252", "", {"T": ["12:30", "13:50"], "R": ["12:30", "13:50"]}, False),
    Course("IST345", "", {"W": ["17:15", "20:05"]}, False),
    Course("IST345", "", {"T": ["9:30", "10:50"], "R": ["9:30", "10:50"]}, False),
    Course("IST345", "", {"T": ["17:15", "20:05"]}, False),
    Course("IST421", "", {"T": ["9:30", "12:15"]}, False),
    Course("IST421", "", {"W": ["9:30", "12:15"]}, False),
    Course("IST421", "", {"R": ["14:00", "16:45"]}, False),
]
labs = [

]

In [144]:
#user preferences
early_time = str(start_widget.value) + ":00"
number_of_classes = no_widget.value
late_time = str(end_widget.value) + ":00"
friday_option = friday_widget.value
max_duration = duration_widget.value

In [145]:
early_time, number_of_classes, late_time, friday_option, max_duration

('9:00', 4, '21:00', False, 158)

In [149]:
#runs all possible combinations of courses based on user request number
all_options = list(combinations(classes, number_of_classes))

new_options = []
for option in all_options:
    for a, b in product(option, option):
        if a != b:
            if run_checks(a, b, early_time, late_time, friday_option, max_duration):
                break
    else:
        new_options.append(option)
final_options = []
for option in new_options:
    check, new_option = check_lab(option)
    if check:
        final_options.append(new_option)

print_schedule(final_options[0])

___________T____________
IST345 09:30:00-10:50:00
ACC252 11:00:00-12:20:00
MAR444 12:30:00-13:50:00
MAR357 17:00:00-18:20:00
___________R____________
IST345 09:30:00-10:50:00
ACC252 11:00:00-12:20:00
MAR444 12:30:00-13:50:00
MAR357 17:00:00-18:20:00
